In [2]:
import wradlib
import matplotlib.pyplot as plt
import pandas as pd
from osgeo import ogr, osr
import numpy as np
#import descartes

In [3]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [4]:
source = wradlib.georef.get_default_projection()
transform = osr.CoordinateTransformation(source, proj)
retransform = osr.CoordinateTransformation(proj, source)

NameError: name 'proj' is not defined

In [ ]:
#dem = wradlib.io.read_generic_netcdf(r"P:\progress\global_radar_map\GMTED2010_15n030_0125deg.nc")

In [3]:
radars = pd.read_csv(r"P:\progress\global_radar_map\GlobalRadarStations.csv", sep = ";", encoding="cp1252")

In [21]:
r = 200000
phi = np.arange(0, 360, 2)
theta = 0.5
bufrs = []
for i in range(len(radars)):
    sitecoords = (radars.x[i], radars.y[i])
    pts = wradlib.georef.polar.spherical_to_proj(
        np.repeat(r, len(phi)), phi, np.repeat(theta, len(phi)), sitecoords
    )
    # Create ring
    ring = ogr.Geometry(ogr.wkbLinearRing)
    for pt in pts:
        ring.AddPoint(pt[0], pt[1])
    # Close ring
    ring.AddPoint(pts[0][0], pts[0][1])

    # Create polygon
    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)
    bufrs.append(poly)

In [63]:
union = bufrs[0]
for i, bufr in enumerate(bufrs[1:]):
    if i in [315,]:
        continue
    union = union.Union(bufr)
#union.Transform(transform)

In [4]:
from osgeo import gdalnumeric

In [15]:
ds = wradlib.io.open_raster(r"E:\data\natural_earth\HYP_50M_SR_W\HYP_50M_SR_W.tif")
# pixel_spacing is in output units (lonlat)
#ds = wradlib.georef.reproject_raster_dataset(ds, projection_target=proj)
rastervalues, rastercoords, proj = wradlib.georef.extract_raster_dataset(ds)
# specify kwargs for plotting, using terrain colormap and LogNorm

In [ ]:
rastervalues

In [17]:
# ... Stack them
rgb = np.dstack([rastervalues[0], rastervalues[1], rastervalues[2]])

# Reshape as list of rgb tuples
color_tuples = np.array([rastervalues[0].flatten(), rastervalues[1].flatten(), rastervalues[2].flatten()]).transpose()

In [18]:
plt.pcolormesh(rastervalues[0], color=color_tuples)

ValueError: RGBA values should be within 0-1 range

In [9]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, aspect='equal')
dem = ax.pcolormesh(rastercoords[..., 0], rastercoords[..., 1],
                    rastervalues, cmap=plt.cm.terrain)

In [ ]:
srcArray = gdalnumeric.LoadFile(r"E:\data\natural_earth\HYP_50M_SR_W\HYP_50M_SR_W.tif")

In [ ]:
plt.imshow(np.dstack((srcArray[0], srcArray[1], srcArray[2])))

In [ ]:
plt.pcolormesh(np.dstack((srcArray[0], srcArray[1], srcArray[2])))

In [28]:
lat = dem["variables"]["latitude"]["data"]
lon = dem["variables"]["longitude"]["data"]
elev = dem["variables"]["elevation"]["data"]

NameError: name 'dem' is not defined

In [ ]:
lon, lat = np.meshgrid(lon, lat)
#lonlat = wradlib.util.gridaspoints(lon, lat)

In [27]:
# Target projection
#   Robinson
proj = wradlib.georef.epsg_to_osr(54030)
#   Mercator world
#proj = wradlib.georef.proj4_to_osr("+proj=merc +lon_0=0 +k=1 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs") 
#   Winkel Tripel
#proj = wradlib.georef.epsg_to_osr(54018)
#proj = wradlib.georef.wkt_to_osr('PROJCS["World_Winkel_Tripel_NGS",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Winkel_Tripel"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",15.0],PARAMETER["Standard_Parallel_1",40.0],UNIT["Meter",1.0]]')

In [ ]:
x, y = wradlib.georef.reproject(lon, lat, projection_target=proj)

In [ ]:
grdlon, grdlat = np.meshgrid(range(-160,180, 20), range(-60,90,30))
grdx, grdy = wradlib.georef.reproject(grdlon, grdlat, projection_target=proj)

In [ ]:
#radpoints = []
#for i in range(len(radars)):
#    point = ogr.Geometry(ogr.wkbPoint)
#    point.AddPoint(radars.x[i], radars.y[i])
#    radpoints.append(point)

In [33]:
def write_geojson(fname, geometry, geom_type):
    """Write GeoJSON file for ogr geometries
    
    Parameters
    ----------
    fname : string
    geometry : ogr Geometry object
    geom_type : ogr Geometry type, 
        e.g. ogr.wkbMultiPolygon, ogr.wkbPolygon, ogr.wkbMultiPoint
    
    """
    outDriver = ogr.GetDriverByName('GeoJSON')

    # Create the output GeoJSON
    outDataSource = outDriver.CreateDataSource(fname)
    outLayer = outDataSource.CreateLayer(fname, geom_type=geom_type)

    # Get the output Layer's Feature Definition
    featureDefn = outLayer.GetLayerDefn()

    # create a new feature
    outFeature = ogr.Feature(featureDefn)

    # Set new geometry
    outFeature.SetGeometry(geometry)

    # Add new feature to output Layer
    outLayer.CreateFeature(outFeature)

    # dereference the feature
    outFeature = None

    # Save and close DataSources
    outDataSource = None

In [ ]:
radpoints = ogr.Geometry(ogr.wkbMultiPoint)
for i in range(len(radars)):
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(radars.x[i], radars.y[i])
    radpoints.AddGeometry(point)
    
radpoints.Transform(transform)

write_geojson("out/radpoints.geojson", radpoints, ogr.wkbMultiPoint)

In [ ]:
bufferDistance = 200000
radbufrs = radpoints.Buffer(bufferDistance)

write_geojson("out/radbufrs.geojson", radbufrs, ogr.wkbMultiPolygon)

In [ ]:
#bufferDistance = 1
#radbufrs = []
#for radpoint in radpoints:
#    poly = radpoint.Buffer(bufferDistance)
#    radbufrs.append(poly)    

In [67]:
# Create ring
ring = ogr.Geometry(ogr.wkbLinearRing)
for i in np.linspace(-89.99, 89.99, 100):
    ring.AddPoint(-179.99, i)
for i in np.linspace(-179.99, 179.99, 100):
    ring.AddPoint(i, 89.99)
for i in np.linspace(89.99, -89.99, 100):
    ring.AddPoint(179.99, i)
for i in np.linspace(179.99, -179.99, 100):
    ring.AddPoint(i, -89.99)

# Create polygon
poly = ogr.Geometry(ogr.wkbPolygon)
poly.AddGeometry(ring)
#poly.Transform(transform)

write_geojson("out/clipfrom.geojson", poly, ogr.wkbPolygon)

In [68]:
#clipped = poly.Difference(radbufrs)
clipped = poly.Difference(union)
#clipped.Transform(retransform)

In [69]:
write_geojson("out/clipped.geojson", clipped, ogr.wkbMultiPolygon)

In [41]:
clipped_dict = eval(clipped.ExportToJson())

In [42]:
from descartes import PolygonPatch
BLUE = '#6699cc'
fig = plt.figure() 
ax = fig.gca() 
ax.add_patch(PolygonPatch(clipped_dict, fc="black", ec="black", alpha=0.5, zorder=2 ))
ax.axis('scaled')

(-10031344.646182792,
 10031344.646182792,
 -9453473.058896221,
 9453473.058896221)

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, aspect="equal")
plt.pcolormesh(x, y, elev, cmap=plt.cm.terrain)
#plt.plot(radars.x, radars.y, "ro")
#ax.add_patch(PolygonPatch(clipped_dict, fc="black", ec="black", alpha=0.7, zorder=2 ))
#ax.axis('scaled')
plt.plot(grdx, grdy, "w+") 